In [3]:
import itertools
from collections import defaultdict

def parse_opcode(opcode):
    """
    ABCDE
     1002

    DE - two-digit opcode,      02 == opcode 2
     C - mode of 1st parameter,  0 == position mode
     B - mode of 2nd parameter,  1 == immediate mode
     A - mode of 3rd parameter,  0 == position mode,
                                      omitted due to being a leading zero
    """
    opcode = str(opcode)
    opcode = '0'*(5-len(opcode)) + opcode
    A = int(opcode[0])
    B = int(opcode[1])
    C = int(opcode[2])
    DE = int(opcode[3:5])
    return DE, C, B, A

def get_final_state(instructions : defaultdict, inputs):
    relative_base_offset = 0
    position = 0
    while True:
        opcode = instructions[position]
        opcode, mode1, mode2, mode3 = parse_opcode(opcode)
        
        # We can determine the location where to find the argument for every opcode... 
        # Also make sure that each argument has a location, even though that might not be used...
        modes = [mode1, mode2, mode3] # Put all modes in a list so we can iterate over them
        arg_locations = [0,0,0]
        for index in range(3):
            if modes[index] == 0: # Position mode
                arg_locations[index] = instructions[position+index+1]
            elif modes[index] == 1: # Direct mode
                arg_locations[index] = position+index+1
            elif modes[index] == 2: # Relative mode
                arg_locations[index] = instructions[position+index+1]+relative_base_offset 
        
        if opcode == 1: 
            instructions[arg_locations[2]] = instructions[arg_locations[0]] + instructions[arg_locations[1]]
            position += 4
        elif opcode == 2: 
            instructions[arg_locations[2]] = instructions[arg_locations[0]] * instructions[arg_locations[1]]
            position += 4
        elif opcode == 3:
            inputted = inputs.pop(0)
            instructions[arg_locations[0]] = inputted
            position += 2
        elif opcode == 4:
            print('yielding ', instructions[arg_locations[0]])
            yield instructions[arg_locations[0]]
            position += 2
        elif opcode == 5: 
            "Opcode 5 is jump-if-true: if the first parameter is non-zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing."
            if instructions[arg_locations[0]]: 
                position = instructions[arg_locations[1]]
            else:
                position += 3
        elif opcode == 6: 
            "Opcode 6 is jump-if-false: if the first parameter is zero, it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing."
            if not instructions[arg_locations[0]]: 
                position = instructions[arg_locations[1]]
            else:
                position += 3
        elif opcode == 7: 
            "Opcode 7 is less than: if the first parameter is less than the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0."
            if instructions[arg_locations[0]] < instructions[arg_locations[1]]:
                instructions[arg_locations[2]] = 1
            else:
                instructions[arg_locations[2]] = 0
            position += 4  
        elif opcode == 8: 
            "Opcode 8 is equals: if the first parameter is equal to the second parameter, it stores 1 in the position given by the third parameter. Otherwise, it stores 0."
            if instructions[arg_locations[0]] == instructions[arg_locations[1]]: 
                instructions[arg_locations[2]] = 1
            else:
                instructions[arg_locations[2]] = 0
            position += 4
        elif opcode == 9:
            """Opcode 9 adjusts the relative base by the value of its only parameter. 
            The relative base increases (or decreases, if the value is negative) by the value of the parameter."""
            relative_base_offset += instructions[arg_locations[0]]
            position += 2
            
        elif opcode == 99:
            break
    print("HALTED")
    raise ValueError("HALTED")


instructions = [int(x) for x in open('inputs/day9.txt').readline().split(',')]

# instructions = [int(x) for x in "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99".split(",")]
# instructions = [int(x) for x in "1102,34915192,34915192,7,4,7,99,0".split(",")]
# instructions = [int(x) for x in "104,1125899906842624,99".split(",")]


## Uncomment for part 1
# temp = defaultdict(int)
# for blaat, instruct in enumerate(instructions):
#     temp[blaat] = instruct

# for i in get_final_state(temp, [1]):
#     print(i)

## Uncomment for part 2
temp = defaultdict(int)
for blaat, instruct in enumerate(instructions):
    temp[blaat] = instruct

for i in get_final_state(temp, [2]):
    print(i)


yielding  83239
83239
HALTED


ValueError: HALTED

In [ ]:

        if opcode in [1,2,4,5,6,7,8, 9]:
            if mode1 == 0: # position mode
                arg1 = instructions[instructions[position+1]]
            elif mode1 == 1: # direct mode
                arg1 = instructions[position+1]
            elif mode1 == 2: # relative mode
                arg1 = instructions[instructions[position+1]+relative_base_offset]
            else:
                raise ValueError("Mode unknown")
                
            if mode2 == 0: # position mode
                arg2 = instructions[instructions[position+2]]
            elif mode2 == 1: # direct mode
                arg2 = instructions[position+2]
            elif mode2 == 2: # relative mode
                arg2 = instructions[instructions[position+2]+relative_base_offset]
            else:
                raise ValueError("Mode unknown")
        
            if mode3 == 0: # position mode
                arg3 = instructions[instructions[position+3]]
            elif mode3 == 1: # direct mode
                arg3 = instructions[position+3]
            elif mode3 == 2: # relative mode
                arg3 = instructions[position+3]+relative_base_offset
            else:
                raise ValueError("Mode unknown")
        